In [1]:
#Importing packages
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import datetime
import seaborn as sns
import math

import joblib

print("Libraries imported!")

Libraries imported!


## Finding similar movies

In [2]:
r_cols = ['user_id', 'movie_id', 'rating']

ratings = pd.read_csv('data/ml-latest-small/ratings.csv')

m_cols = ['movie_id', 'title']
movies = pd.read_csv('data/ml-latest-small/movies.csv')

ratings_df = pd.merge(movies, ratings)

ratings_df = ratings_df[['movieId', 'title', 'userId', 'rating']]

ratings_df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [3]:
ratings_df[ratings_df.title.str.contains('Pulp Fiction')]

,movieId,title,userId,rating
7860,296,Pulp Fiction (1994),1,3.0
7861,296,Pulp Fiction (1994),4,1.0
7862,296,Pulp Fiction (1994),5,5.0
7863,296,Pulp Fiction (1994),6,2.0
7864,296,Pulp Fiction (1994),8,4.0
...,...,...,...,...
8163,296,Pulp Fiction (1994),607,3.0
8164,296,Pulp Fiction (1994),608,5.0
8165,296,Pulp Fiction (1994),609,4.0
8166,296,Pulp Fiction (1994),610,5.0


In [4]:
ratings_df.userId.max()

793

In [5]:
ratings_df_coumns = ['movieId', 'title', 'userId', 'rating']

In [6]:
PF_movieId = ratings_df[ratings_df.title.str.contains('Pulp Fiction')].movieId.values[0]
PF_movieId

296

In [7]:
PF_title = ratings_df[ratings_df.title.str.contains('Pulp Fiction')].title.values[0]
PF_title

'Pulp Fiction (1994)'

In [8]:
ratings_df_cloned = ratings_df

In [9]:
ratings_df_cloned.loc[len(ratings_df_cloned)] = [PF_movieId, PF_title, 7793, 5]

In [10]:
ratings_df_cloned

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5
...,...,...,...,...
100835,193583,No Game No Life: Zero (2017),184,3.5
100836,193585,Flint (2017),184,3.5
100837,193587,Bungo Stray Dogs: Dead Apple (2018),184,3.5
100838,193609,Andrew Dice Clay: Dice Rules (1991),331,4.0


In [11]:
movie_ratings = ratings_df.pivot_table(index=['userId'], columns=['title'], values='rating')

movie_ratings.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
pulp_fiction_ratings = movie_ratings['Pulp Fiction (1994)']
pulp_fiction_ratings.head()

userId
1    3.0
2    NaN
3    NaN
4    1.0
5    5.0
Name: Pulp Fiction (1994), dtype: float64

In [13]:
similar_movies = movie_ratings.corrwith(pulp_fiction_ratings)
similar_movies.dropna(inplace=True)
df = pd.DataFrame(similar_movies)
print(df.shape)
df.head(10)

(4878, 1)


,0
title,
"'burbs, The (1989)",-0.187323
(500) Days of Summer (2009),-0.044034
*batteries not included (1987),-0.738549
...And Justice for All (1979),-0.188982
10 Cent Pistol (2015),1.000000
10 Cloverfield Lane (2016),-0.201347
10 Things I Hate About You (1999),-0.126206
"10,000 BC (2008)",-0.409514
101 Dalmatians (1996),0.289800


Let's sort the results and see the similar movies....

In [14]:
similar_movies.sort_values(ascending=False).head(30)

title
Four Feathers, The (2002)                                                     1.0
After Life (Wandafuru raifu) (1998)                                           1.0
Necessary Roughness (1991)                                                    1.0
Wings of the Dove, The (1997)                                                 1.0
Thieves (Voleurs, Les) (1996)                                                 1.0
Somebody to Love (1994)                                                       1.0
When in Rome (2010)                                                           1.0
Brave One, The (2007)                                                         1.0
Tuck Everlasting (2002)                                                       1.0
Distinguished Gentleman, The (1992)                                           1.0
Maxed Out: Hard Times, Easy Credit and the Era of Predatory Lenders (2006)    1.0
Wolf Children (Okami kodomo no ame to yuki) (2012)                            1.0
Looking fo

The results don't make sense...:( Let's think about it for a moment... <br />
There are people who watched Pulp Fiction (PF) and other obscure movies that don't align with our interests. Clearly those recommendations would be good for them, but not for all of us. So let's dig deep dive into it.

First, let's get rid of movies that were only watched by a few people. Very small sample of ratings would skew our results.

In [15]:
movie_stats = ratings_df.groupby('title').agg({'rating': [np.size, np.mean]})
movie_stats

rating          
                                            size      mean
title                                                     
'71 (2014)                                   1.0  4.000000
'Hellboy': The Seeds of Creation (2004)      1.0  4.000000
'Round Midnight (1986)                       2.0  3.500000
'Salem's Lot (2004)                          1.0  5.000000
'Til There Was You (1997)                    2.0  4.000000
...                                          ...       ...
eXistenZ (1999)                             22.0  3.863636
xXx (2002)                                  24.0  2.770833
xXx: State of the Union (2005)               5.0  2.000000
¡Three Amigos! (1986)                       26.0  3.134615
À nous la liberté (Freedom for Us) (1931)    1.0  1.000000

[9719 rows x 2 columns]

In [16]:
movie_stats[movie_stats['rating']['size'] >= 100].sort_values([('rating', 'mean')], ascending=False).head()

rating          
                                   size      mean
title                                            
Shawshank Redemption, The (1994)  317.0  4.429022
Godfather, The (1972)             192.0  4.289062
Fight Club (1999)                 219.0  4.276256
Godfather: Part II, The (1974)    129.0  4.259690
Departed, The (2006)              107.0  4.252336

In [17]:
df = movie_stats[movie_stats['rating']['size'] >= 50].join(pd.DataFrame(similar_movies, columns=['similarity']))
df

,"(rating, size)","(rating, mean)",similarity
title,,,
10 Things I Hate About You (1999),54.0,3.527778,-0.126206
12 Angry Men (1957),57.0,4.149123,-0.048795
2001: A Space Odyssey (1968),109.0,3.894495,0.117497
28 Days Later (2002),58.0,3.974138,-0.008989
300 (2007),80.0,3.681250,0.192834
...,...,...,...
X2: X-Men United (2003),76.0,3.723684,-0.011742
You've Got Mail (1998),50.0,3.120000,-0.057161
Young Frankenstein (1974),69.0,3.992754,0.075178


In [18]:
df.sort_values(['similarity'], ascending=False)[1:6]

,"(rating, size)","(rating, mean)",similarity
title,,,
"Wolf of Wall Street, The (2013)",54.0,3.916667,0.579915
Fight Club (1999),219.0,4.276256,0.545278
Kill Bill: Vol. 1 (2003),131.0,3.961832,0.504147
Interstellar (2014),73.0,3.993151,0.503411
True Romance (1993),65.0,3.800000,0.484209


## Item-based Collaborative Filtering

In [19]:
ratings_df_cloned

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5
...,...,...,...,...
100835,193583,No Game No Life: Zero (2017),184,3.5
100836,193585,Flint (2017),184,3.5
100837,193587,Bungo Stray Dogs: Dead Apple (2018),184,3.5
100838,193609,Andrew Dice Clay: Dice Rules (1991),331,4.0


In [20]:
ratings_df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [21]:
user_ratings = ratings_df.pivot_table(index=['userId'], columns=['title'], values='rating')
user_ratings.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
corr_matrix = user_ratings.corr()
corr_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
corr_matrix = user_ratings.corr(method='pearson', min_periods=100)
corr_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx: State of the Union (2005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
my_rating = {"Pulp Fiction (1994)": 5, "Goodfellas (1990)" : 5}

In [25]:
user_ratings = ratings_df_cloned.pivot_table(index=['userId'], columns=['title'], values='rating')
user_ratings.loc[7793]

title
'71 (2014)                                  NaN
'Hellboy': The Seeds of Creation (2004)     NaN
'Round Midnight (1986)                      NaN
'Salem's Lot (2004)                         NaN
'Til There Was You (1997)                   NaN
                                             ..
eXistenZ (1999)                             NaN
xXx (2002)                                  NaN
xXx: State of the Union (2005)              NaN
¡Three Amigos! (1986)                       NaN
À nous la liberté (Freedom for Us) (1931)   NaN
Name: 7793, Length: 9719, dtype: float64

In [26]:
user_ratings.loc[7793].dropna()

title
Pulp Fiction (1994)    5.0
Name: 7793, dtype: float64

In [27]:
corr_matrix = user_ratings.corr()
corr_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,NaN,-0.067522,NaN,-0.388883,1.000000,...,NaN,NaN,NaN,NaN,NaN,1.000000,0.762493,NaN,-0.337963,NaN
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.831858,NaN,...,NaN,-0.595769,-1.0,NaN,NaN,0.762493,1.000000,0.94388,0.500000,NaN
xXx: State of the Union (2005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,...,NaN,-0.755929,NaN,NaN,NaN,NaN,0.943880,1.00000,NaN,NaN


In [28]:
corr_matrix = user_ratings.corr(method='pearson', min_periods=100)
corr_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx: State of the Union (2005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
my_ratings = user_ratings.loc[100].dropna()
my_ratings

title
12 Angry Men (1957)                            4.0
8 Mile (2002)                                  4.0
About Last Night... (1986)                     4.0
Ace Ventura: When Nature Calls (1995)          1.0
Affair to Remember, An (1957)                  4.0
                                              ... 
William Shakespeare's Romeo + Juliet (1996)    4.0
Witness (1985)                                 3.0
Wizard of Oz, The (1939)                       4.5
Working Girl (1988)                            4.5
Young Frankenstein (1974)                      4.5
Name: 100, Length: 148, dtype: float64

In [30]:
sim_candidates = pd.Series()

for i in range(0, len(my_ratings.index)):
    print("Adding similarity for {} ...".format(my_ratings.index[i]))
    #Retrive similar movies for the one that I rated
    sims = corr_matrix[my_ratings.index[i]].dropna()
    #Scale its similarity by how well I rated this movie
    sims = sims.map(lambda x: x * my_ratings[i])
    #Add the score to the list of similarity candidates
    sim_candidates = sim_candidates.append(sims)
    
print("sorting...")
sim_candidates.sort_values(inplace=True, ascending=False)
sim_candidates.head(10)

Adding similarity for 12 Angry Men (1957) ...
Adding similarity for 8 Mile (2002) ...
Adding similarity for About Last Night... (1986) ...
Adding similarity for Ace Ventura: When Nature Calls (1995) ...
Adding similarity for Affair to Remember, An (1957) ...
Adding similarity for Amadeus (1984) ...
Adding similarity for American History X (1998) ...
Adding similarity for American Pie (1999) ...
Adding similarity for American President, The (1995) ...
Adding similarity for American in Paris, An (1951) ...
Adding similarity for Analyze This (1999) ...
Adding similarity for As Good as It Gets (1997) ...
Adding similarity for Austin Powers: International Man of Mystery (1997) ...
Adding similarity for Austin Powers: The Spy Who Shagged Me (1999) ...
Adding similarity for Back to the Future (1985) ...
Adding similarity for Batman (1989) ...
Adding similarity for Beautiful Mind, A (2001) ...
Adding similarity for Bed of Roses (1996) ...
Adding similarity for Benny & Joon (1993) ...
Adding si

Breakfast Club, The (1985)            4.5
Catch Me If You Can (2002)            4.5
Sleepless in Seattle (1993)           4.5
Pretty Woman (1990)                   4.5
American History X (1998)             4.0
Batman (1989)                         4.0
Dances with Wolves (1990)             4.0
Finding Nemo (2003)                   4.0
Forrest Gump (1994)                   4.0
Four Weddings and a Funeral (1994)    4.0
dtype: float64

In [31]:
sim_candidates = sim_candidates.groupby(sim_candidates.index).sum()
sim_candidates.sort_values(inplace=True, ascending=False)
sim_candidates.head(10)


Forrest Gump (1994)                     17.273811
Jurassic Park (1993)                    11.579480
Terminator 2: Judgment Day (1991)       10.126700
Independence Day (a.k.a. ID4) (1996)     9.499573
Matrix, The (1999)                       8.918065
True Lies (1994)                         7.756792
Braveheart (1995)                        7.266478
Dances with Wolves (1990)                7.249983
Silence of the Lambs, The (1991)         7.118815
Pulp Fiction (1994)                      6.981229
dtype: float64

In [32]:
sim_candidates[3:3+10]

Independence Day (a.k.a. ID4) (1996)                     9.499573
Matrix, The (1999)                                       8.918065
True Lies (1994)                                         7.756792
Braveheart (1995)                                        7.266478
Dances with Wolves (1990)                                7.249983
Silence of the Lambs, The (1991)                         7.118815
Pulp Fiction (1994)                                      6.981229
Back to the Future (1985)                                6.894356
Saving Private Ryan (1998)                               6.556293
Star Wars: Episode V - The Empire Strikes Back (1980)    6.145828
dtype: float64

In [33]:
ratings_df[ratings_df.title.str.contains('Help, The')]

,movieId,title,userId,rating
93310,88810,"Help, The (2011)",10,3.0
93311,88810,"Help, The (2011)",47,3.5
93312,88810,"Help, The (2011)",68,4.5
93313,88810,"Help, The (2011)",73,4.0
93314,88810,"Help, The (2011)",98,4.0
93315,88810,"Help, The (2011)",103,3.5
93316,88810,"Help, The (2011)",105,3.5
93317,88810,"Help, The (2011)",153,4.0
93318,88810,"Help, The (2011)",159,3.5
93319,88810,"Help, The (2011)",177,4.0


In [34]:
r_cols = ['user_id', 'movie_id', 'rating']

ratings = pd.read_csv('data/ml-latest-small/ratings.csv')

m_cols = ['movie_id', 'title']
movies = pd.read_csv('data/ml-latest-small/movies.csv')

links = pd.read_csv('data/ml-latest-small/links.csv')

tags = pd.read_csv('data/ml-latest-small/tags.csv')

ratings_df = pd.merge(movies, ratings)

#ratings_df = ratings_df[['movieId', 'title', 'userId', 'rating']]

print(ratings_df.shape)

ratings_df.head()

(100839, 6)


,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [35]:
pd.merge(ratings_df, links).to_csv('ratings_100k.csv')

In [36]:
df = pd.read_csv('ratings_100k.csv', index_col=0)
df['year'] = df.title.apply(lambda x: x[-5:-1])

other_year = df['year'].value_counts()[df['year'].value_counts() <= 4].index
df['year'] = df['year'].replace(other_year, np.nan)

df.dropna(inplace=True)


In [37]:
for i in range(5):
    df['sub_genre_{}'.format(i+1)] = df['genres'].apply(lambda x: x.split('|')[i] if len(x.split('|')) > i else None)
    
df.head()

,movieId,title,genres,userId,rating,timestamp,imdbId,tmdbId,year,sub_genre_1,sub_genre_2,sub_genre_3,sub_genre_4,sub_genre_5
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,114709,862.0,1995,Adventure,Animation,Children,Comedy,Fantasy
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,114709,862.0,1995,Adventure,Animation,Children,Comedy,Fantasy
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,114709,862.0,1995,Adventure,Animation,Children,Comedy,Fantasy
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,114709,862.0,1995,Adventure,Animation,Children,Comedy,Fantasy
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,114709,862.0,1995,Adventure,Animation,Children,Comedy,Fantasy


In [38]:
sub_genre_1_count = pd.DataFrame(df.sub_genre_1.value_counts().sort_index())
sub_genre_1_count.index

Index(['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')

In [39]:
sub_genre_2_count = pd.DataFrame(df.sub_genre_2.value_counts().sort_index())
sub_genre_2_count

,sub_genre_2
Adventure,13814
Animation,3757
Children,3324
Comedy,7566
Crime,7652
Documentary,179
Drama,17875
Fantasy,3182
Film-Noir,397
Horror,3484


In [40]:
list_of_genre = ['(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [41]:
def movie_identity(title):
    title = title.title()
    print('The movie title is {}'.format(df[df.title.str.contains(title)].title.values[1]))
    print('The movieId is {}'.format(df[df.title.str.contains(title)].movieId.values[1]))
    print('The genre(s) is(are) {}'.format(df[df.title.str.contains(title)].genres.values[1]))

In [42]:
movie_identity('fight club')

The movie title is Fight Club (1999)
The movieId is 2959
The genre(s) is(are) Action|Crime|Drama|Thriller


In [43]:
sub_genre_3_count = pd.DataFrame(df.sub_genre_3.value_counts().sort_index())
len(sub_genre_3_count)

17

In [44]:
sub_genre_4_count = pd.DataFrame(df.sub_genre_4.value_counts().sort_index())
len(sub_genre_4_count)

16

In [45]:
sub_genre_5_count = pd.DataFrame(df.sub_genre_5.value_counts().sort_index())
len(sub_genre_5_count)

14

In [46]:
df.genres.value_counts()

Comedy                                    7193
Drama                                     6289
Comedy|Romance                            3967
Comedy|Drama|Romance                      3000
Comedy|Drama                              2850
                                          ... 
Action|Horror|Mystery|Sci-Fi                 1
Crime|Drama|Fantasy                          1
Action|Adventure|Drama|Fantasy|Romance       1
Action|Comedy|Horror|Musical                 1
Animation|Drama|Fantasy|Mystery              1
Name: genres, Length: 950, dtype: int64

In [47]:
total_df = pd.DataFrame(columns={'Total movies'})
total_df

,Total movies


In [48]:
# total_df.loc['1'] = 5
# total_df

In [49]:
list(sub_genre_1_count.index)

['(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [50]:
for genre in list_of_genre:
    
    total_df.loc[genre] = 0

In [51]:
for genre in list_of_genre:
    
    if genre in list(sub_genre_1_count.index):
        
        total_df.loc[genre] += sub_genre_1_count.loc[genre].sub_genre_1
    
    if genre in list(sub_genre_2_count.index):
        
        total_df.loc[genre] += sub_genre_2_count.loc[genre].sub_genre_2
        
    if genre in list(sub_genre_3_count.index):
        
        total_df.loc[genre] += sub_genre_3_count.loc[genre].sub_genre_3
    
    if genre in list(sub_genre_4_count.index):
        
        total_df.loc[genre] += sub_genre_4_count.loc[genre].sub_genre_4

    if genre in list(sub_genre_5_count.index):
        
        total_df.loc[genre] += sub_genre_5_count.loc[genre].sub_genre_5

total_df

,Total movies
(no genres listed),38
Action,30624
Adventure,24153
Animation,6981
Children,9207
Comedy,39044
Crime,16677
Documentary,1218
Drama,41921
Fantasy,11651


In [52]:
sub_genre_4_count.loc['Drama'].sub_genre_4 + sub_genre_5_count.loc['Drama'].sub_genre_5

1222

In [53]:
total_df.to_csv("total_movies_by_genre.csv")

In [54]:
df = df[['movieId', 'title', 'year', 'userId', 'rating', 'sub_genre_1', 'sub_genre_2', 'sub_genre_3', 'sub_genre_4', 'sub_genre_5']]

In [55]:
df.to_csv('ratings_100k.csv')

In [56]:
total_df

,Total movies
(no genres listed),38
Action,30624
Adventure,24153
Animation,6981
Children,9207
Comedy,39044
Crime,16677
Documentary,1218
Drama,41921
Fantasy,11651


In [57]:
avg_rating_by_genre = df.pivot_table(index=['sub_genre_1'], values="rating")
avg_rating_by_genre

,rating
sub_genre_1,
(no genres listed),3.421053
Action,3.448031
Adventure,3.566302
Animation,3.577542
Children,3.153553
Comedy,3.394197
Crime,3.834544
Documentary,3.829389
Drama,3.647336


In [58]:
for genre in list(avg_rating_by_genre.index):
    
    if genre in list(df.pivot_table(index=['sub_genre_2'], values="rating").index):
        
        avg_rating_by_genre.loc[genre].rating += df.pivot_table(index=['sub_genre_2'], values="rating").loc[genre].rating
        avg_rating_by_genre.loc[genre].rating *= 0.5

    if genre in list(df.pivot_table(index=['sub_genre_3'], values="rating").index):
        
        avg_rating_by_genre.loc[genre].rating += df.pivot_table(index=['sub_genre_3'], values="rating").loc[genre].rating
        avg_rating_by_genre.loc[genre].rating *= 0.5
        
    if genre in list(df.pivot_table(index=['sub_genre_4'], values="rating").index):
        
        avg_rating_by_genre.loc[genre].rating += df.pivot_table(index=['sub_genre_4'], values="rating").loc[genre].rating
        avg_rating_by_genre.loc[genre].rating *= 0.5
        
    if genre in list(df.pivot_table(index=['sub_genre_5'], values="rating").index):
        
        avg_rating_by_genre.loc[genre].rating += df.pivot_table(index=['sub_genre_5'], values="rating").loc[genre].rating
        avg_rating_by_genre.loc[genre].rating *= 0.5
        

In [59]:
avg_rating_by_genre

,rating
sub_genre_1,
(no genres listed),3.421053
Action,3.448031
Adventure,3.516121
Animation,3.614310
Children,3.519076
Comedy,3.561137
Crime,3.528966
Documentary,3.835022
Drama,3.602882


In [60]:
df

,movieId,title,year,userId,rating,sub_genre_1,sub_genre_2,sub_genre_3,sub_genre_4,sub_genre_5
0,1,Toy Story (1995),1995,1,4.0,Adventure,Animation,Children,Comedy,Fantasy
1,1,Toy Story (1995),1995,5,4.0,Adventure,Animation,Children,Comedy,Fantasy
2,1,Toy Story (1995),1995,7,4.5,Adventure,Animation,Children,Comedy,Fantasy
3,1,Toy Story (1995),1995,15,2.5,Adventure,Animation,Children,Comedy,Fantasy
4,1,Toy Story (1995),1995,17,4.5,Adventure,Animation,Children,Comedy,Fantasy
...,...,...,...,...,...,...,...,...,...,...
100834,193581,Black Butler: Book of the Atlantic (2017),2017,184,4.0,Action,Animation,Comedy,Fantasy,None
100835,193583,No Game No Life: Zero (2017),2017,184,3.5,Animation,Comedy,Fantasy,None,None
100836,193585,Flint (2017),2017,184,3.5,Drama,None,None,None,None
100837,193587,Bungo Stray Dogs: Dead Apple (2018),2018,184,3.5,Action,Animation,None,None,None
